# Google Maps Review Responder

This Python app automates the process of fetching and responding Google Maps reviews for any business or location.

## Overview
The app performs two main tasks:
1. **Scrape Reviews** – Uses a web scraping script to extract reviews directly from Google Maps.
2. **Respond Content** – Leverages OpenAI or Claude language models to generate concise, insightful summaries of the collected reviews and analyse the sentiments.

## Tech Stack
- **Python** – Core language
- **Playwright** – For scraping reviews
- **OpenAI API** – For natural language summarization
- **Jupyter Notebook** – For exploration, testing, and demonstration

### Credits
The web scraping logic is **inspired by [Antonello Zanini’s blog post](https://blog.apify.com/how-to-scrape-google-reviews/)** on building a Google Reviews scraper. Special thanks for the valuable insights on **structuring and automating the scraping workflow**, which greatly informed the development of this improved scraper.

This app, however, uses an **enhanced version of the scraper** that can scroll infinitely to load more reviews until it collects **at least 1,000 reviews**. If only a smaller number of reviews are available, the scraper stops scrolling earlier.

**Note:** This project is intended for educational and research purposes. Please ensure compliance with Google’s [Terms of Service](https://policies.google.com/terms) when scraping or using their data.


In [ ]:
#Make sure pip is available and up to date inside the venv
!python3 -m ensurepip --upgrade

#Verify that pip now points to the venv path (should end with /.venv/bin/pip)
!which pip3

#Install Playwright inside the venv
!pip3 install playwright

#Download the required browser binaries and dependencies
!python3 -m playwright install

# the following not not needed

# Install the dotenv package
# !pip3 install python-dotenv

# Install the openai package
# !pip3 install openai

Looking in links: /var/folders/n9/bqzhr3vs6rsc6v_m5w8bz3b40000gn/T/tmpuu7le6om
/Users/jaechoi/repos/llm_replyGenix/.venv/bin/pip3


In [2]:
import asyncio
from playwright.async_api import async_playwright
from IPython.display import Markdown, display
import os
from dotenv import load_dotenv
from openai import OpenAI


In [11]:
# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')
# api_key = os.getenv('ANTHROPIC_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


API key found and looks good so far!


In [19]:
async def scrape_google_reviews(url):
    # Where to store the scraped data
    reviews = []

    async with async_playwright() as p:
         # Initialize a new Playwright instance
        browser = await p.chromium.launch(
            headless=True  # Set to False if you want to see the browser in action
        )
        context = await browser.new_context()
        page = await context.new_page()

        # The URL of the Google Maps reviews page

        # Navigate to the target Google Maps page
        print("Navigating to Google Maps page...")
        await page.goto(url)

        # Wait for initial reviews to load
        print("Waiting for initial reviews to load...")
        review_html_elements = page.locator("div[data-review-id][jsaction]")
        await review_html_elements.first.wait_for(state="visible", timeout=10000)
              
        # Iterate over the elements and scrape data from each of them
        for review_html_element in await review_html_elements.all():
            try:
                # Scraping logic
                user_contrib_element = review_html_element.locator("button[data-href*=\"/maps/contrib/\"]:not([aria-label])")
                user_url = await user_contrib_element.get_attribute("data-href")

                username_element = user_contrib_element.locator("div").first
                username = await username_element.text_content()

                stars_element = review_html_element.locator("[aria-label*=\"star\"]")
                stars_label = await stars_element.get_attribute("aria-label")

                # Extract the review score from the stars label
                stars = None
                for i in range(1, 6):
                    if stars_label and str(i) in stars_label:
                        stars = i
                        break

                # Get the next sibling of the previous element with an XPath expression
                time_sibling = stars_element.locator("xpath=following-sibling::span")
                time = await time_sibling.text_content()

                # Select the "More" button and if it is present, click it
                more_element = review_html_element.locator("button[aria-label=\"See more\"]")
                if await more_element.count() > 0:
                    await more_element.click()

                text_element = review_html_element.locator("div[tabindex=\"-1\"][id][lang]")
                text = await text_element.text_content()

                # Populate a new object with the scraped data and append it to the list
                review = {
                    "user_url": user_url,
                    "username": username,
                    "stars": stars,
                    "time": time,
                    "text": text
                }
                reviews.append(review)
                    
            except Exception as e:
                print(f"Error scraping review : {str(e)}")
                continue

        print(f"\nSuccessfully scraped {len(reviews)} reviews!")

        # Close the browser and release its resources
        await browser.close()

        return reviews

In [17]:
test_url = "https://www.google.com/maps/place/Costco+Wholesale/@33.0355379,-96.8624758,14.29z/data=!4m8!3m7!1s0x864c23810890f0df:0xf9f33301a96d2ff4!8m2!3d33.0238859!4d-96.8314151!9m1!1b1!16s%2Fg%2F1tj58551?entry=ttu&g_ep=EgoyMDI1MTEwOS4wIKXMDSoASAFQAw%3D%3D"

In [ ]:
# results = await scrape_google_reviews(test_url)
# print(results)

Navigating to Google Maps page...
Waiting for initial reviews to load...

Successfully scraped 8 reviews!
[{'user_url': 'https://www.google.com/maps/contrib/109703395521871881723/reviews?hl=en-US', 'username': 'Ameen Shawwa', 'stars': 4, 'time': 'a month ago', 'text': 'I\'m a Costco guy. Of course I\'m gonna leave a review for Costco. But in all seriousness, this is my four-star review for Costco, and in all honesty, it has served quite well for the past 15 years. I\'ve been going here with my family literally every weekend. We get stuff for the house or sometimes for work, who knows, but it\'s been every week consistently for 15 years.\n\nI gotta be honest; it\'s always pretty solid. They always have the coolest stuff in stock, and we always get the best deals. We rarely run out of anything because we keep getting stuff from here. The workers are usually the coolest, but it\'s not perfect because of, well, literally the workers.\n\nThe workers here sometimes take their job too serious

In [49]:
system_prompt = """
You are an expert assistnat that reply to google reviews,
Provide replies to given reviews. Keep them under 3 sentences.
Respond in markdonw. Do not wrap the markdown in a code block - reply just with the markdown.
"""

In [54]:
# Define our user prompt

user_prompt_prefix = """
Here are the reviews of a google map location/business.
Provide replies of the given eviews. Keep them under 3 sentenses.
To a negative review, be apologetic, to a positive review, thank it.
"""

In [51]:
async def respond_to_reviews(url):
    openai = OpenAI()
    responses = []
    reviews = await scrape_google_reviews(url)
    
    print("Generating replies.")
    for review in reviews:
        prompt = user_prompt_prefix
        prompt += f"User: {review['username']}\n"
        prompt += f"Review: {review['text']}"
        response = openai.chat.completions.create(
            model = "gpt-5-nano",
            messages=[{"role": "user", "content": prompt}]
            # max_tokens=80
        )
        # Fix: Access the response content correctly
        response_text = response.choices[0].message.content.strip()
        
        # Fix: Typo in 'response' key
        responses.append({'review': review, 'response': response_text})
    return responses

In [52]:
url = "https://www.google.com/maps/place/Costco+Wholesale/@33.0355379,-96.8624758,14.29z/data=!4m8!3m7!1s0x864c23810890f0df:0xf9f33301a96d2ff4!8m2!3d33.0238859!4d-96.8314151!9m1!1b1!16s%2Fg%2F1tj58551?entry=ttu&g_ep=EgoyMDI1MTEwOS4wIKXMDSoASAFQAw%3D%3D"
results = await respond_to_reviews(url)


Navigating to Google Maps page...
Waiting for initial reviews to load...

Successfully scraped 8 reviews!
Generating replies.


In [53]:
import pprint
pprint.pprint(results)


[{'response': 'Thank you, Ameen, for 15 years of loyalty and for the kind '
              'words about our selection and staff. We’re sorry the membership '
              'checks and the card-issue caused frustration—that feedback '
              'helps us improve. If you’d like, please reach out with details '
              'so we can look into it further.',
  'review': {'stars': 4,
             'text': "I'm a Costco guy. Of course I'm gonna leave a review for "
                     'Costco. But in all seriousness, this is my four-star '
                     'review for Costco, and in all honesty, it has served '
                     "quite well for the past 15 years. I've been going here "
                     'with my family literally every weekend. We get stuff for '
                     "the house or sometimes for work, who knows, but it's "
                     'been every week consistently for 15 years.\n'
                     '\n'
                     "I gotta be honest; it's a